<a href="https://colab.research.google.com/github/csar95/hugging-face-course/blob/main/workbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import openai
from pytube import YouTube

# Extraction of the audio of a YouTube video

In [3]:
# Ask user for the YouTube video URL
yt_url = "https://www.youtube.com/watch?v=VRm2UMsFVz0&list=PLbUIzNzCKD8CUVjPwFAUoNDc0souCNHLs&index=1"

# Create a YouTube object from the URL
yt = YouTube(yt_url)

In [4]:
# Get the audio stream
audio_stream = yt.streams.filter(only_audio=True).first()

In [5]:
# Download the audio stream
output_path = "data"
filename = f"{yt.title}.mp3"
audio_stream.download(output_path=output_path, filename=filename)

'/Users/csr95/Desktop/Mis_Documentos/Code_Python/whispering-key-points/data/ACID Transactions.mp3'

# OpenAI API

In [2]:
# Load your API key from an environment variable or secret management service
openai.api_key = os.getenv("OPENAI_API_KEY")

## Whisper

In [6]:
language = 'en'

In [7]:
%%time

with open(f"{output_path}/{filename}", "rb") as audio_file:
    transcript = openai.Audio.transcribe("whisper-1", audio_file, language)

CPU times: user 21.8 ms, sys: 8.21 ms, total: 30 ms
Wall time: 17.3 s


In [8]:
transcript.text

"Hello, my name is Niklas and today we're going to talk about transactions in a relational database management system. More specifically, we're going to talk about the ACID acronym, what it stands for and what it actually means. So fundamentally, a transaction is a unit of work executed to retrieve, insert, remove and or update data. And this unit of work typically consists of running one or more statements or queries against some kind of database. In a relational database management system, transactions come with specific properties that serve as guarantees of sorts. So we can rely on the fact that all transactions in a relational database management system will be atomic, they will be consistent, they will be isolated and they will be durable or ACID for short. It's worth mentioning that while all relational database management systems have these guarantees, not all database management systems do so. So some have base properties instead of ACID, but we won't go into this in this vide

## ChatGPT

In [13]:
prompt = "Summarize the following text:"

In [14]:
content = f"{prompt} {transcript.text}"

In [16]:
res = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                   messages=[{"role": "system", "content": "You are ChatGPT, a large language model trained by OpenAI. Answer as concisely as possible."},
                                             {"role": "user", "content": content}])

In [22]:
res.choices[0].message.content

'The article discusses transactions in a relational database management system, focusing on the ACID acronym which guarantees that all transactions will be atomic, consistent, isolated and durable. The writer uses the example of a bank transfer to illustrate the importance of these guarantees. Atomicity ensures that all queries and operations must run successfully or none of them will be committed. Consistency means that after the transaction, all data is still in a valid state. Isolation means that transactions are executed sequentially, and durability ensures that changes are committed permanently in the event of a catastrophic failure.'

# Hugging Face models

In [39]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
checkpoint = "facebook/bart-large-cnn"
transcript = "Hello, my name is Niklas and today we're going to talk about transactions in a relational database management system. More specifically, we're going to talk about the ACID acronym, what it stands for and what it actually means. So fundamentally, a transaction is a unit of work executed to retrieve, insert, remove and or update data. And this unit of work typically consists of running one or more statements or queries against some kind of database. In a relational database management system, transactions come with specific properties that serve as guarantees of sorts. So we can rely on the fact that all transactions in a relational database management system will be atomic, they will be consistent, they will be isolated and they will be durable or ACID for short. It's worth mentioning that while all relational database management systems have these guarantees, not all database management systems do so. So some have base properties instead of ACID, but we won't go into this in this video. A common example used to illustrate a transaction is a bank transfer. So in this use case, we want to transfer money from one bank account into another. And for this example, we're going to assume that all the bank account data is being persisted in a single database. And so in order to do this, we'll first need to make sure that there are sufficient funds on the payer's account. Then we need to subtract that amount from the payer's account, add it to the receiver's account. And finally, we want to mark the transaction as successful. And we can do this by executing four queries against the database. So first, we're going to make a select to check the funds, followed by three update statements for the rest of the logic subtraction, adding and finally marking the transfer as successful. How to group the queries into different transactions is up to the application programmer. So we could perform each query in its own transaction, but for reasons that we'll soon see, this is probably a bad idea in our example. And it would probably make more sense to think of all the four queries as being part of one logical transaction and then implement the application logic accordingly. So let's go back to the ACID properties in the context of this example, starting with the A for atomic. Simply put, atomicity in the context of a transaction means that either all the queries and operations are run, or none of them are. This guarantees that if there is a failure at any point during the transaction, none of the statements are actually committed. Going back to our example, it's evident why this guarantee by the database management system is so useful for us. Imagine that we run into an error during the transaction. The first two queries have run successfully here, and the money has been deducted from the payer's account. However, we have failed to add the money to the receiver's account or mark the transfer as successful. So if the successful queries were actually to be committed to the database, regardless, we would be in a worrying situation. The status of the transfer would not change, and it would appear the money had just vanished into thin air. We have the guarantee of atomicity of transactions, however. We can indeed group all the statements under one transaction and rely on the database management system to ensure that if anything goes wrong during the transaction, none of the changes are committed. So in this case, the money wouldn't be taken from the payer's account. Next, we have C for consistency. By consistency, we mean that after a transaction is performed, all of the data is still in a valid state. In the context of our example, we could set a constraint, for example, on the account balance field to make sure that it never goes below zero. So after that, we could rely on the database management system not to commit a transaction that would leave the database in a state with negative numbers in the account balance field. Or in other words, transfer money that isn't there. For the third property, I for isolated, means that the statements are executed in a seemingly consecutive or sequential way. So at the highest isolation level, as far as the application programmer knows, each transaction is run after the previous one is fully completed. So in practice, database management system might execute transactions concurrently, but they do make the guarantee that the result is the same as it would be in a more consecutive approach. Lowering the isolation level means that transactions which are being run concurrently by the database management system might actually affect each other and see each other at different steps of the way. There is always a trade-off between the guarantees of a sequential approach at a high isolation level compared to the performance gains that we get if we lower the isolation level, then we're going to lose some of the guarantees of isolation. And finally, the last property of an ACID transaction is durability. Durability means that once commitment has been acknowledged by the database management system, we have a guarantee that the changes have been committed permanently and will persist in the event of a power outage or any other catastrophic failure. In practice, this means that changes have always been flushed from any potential buffers in memory and have been written to disk when the application is informed that the transaction has been committed. And that was my quick run-through of ACID transactions. I hope the video was of some use to you and thanks for listening. Until next time!"

## Alternative with the pipeline function

We have to truncate the input first using the tokenizer.

In [4]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [47]:
inputs = tokenizer(transcript, truncation=True, return_tensors="pt")
print(len(inputs['input_ids'][0]))
truncated_input = tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True)
truncated_input

1024


"Hello, my name is Niklas and today we're going to talk about transactions in a relational database management system. More specifically, we're going to talk about the ACID acronym, what it stands for and what it actually means. So fundamentally, a transaction is a unit of work executed to retrieve, insert, remove and or update data. And this unit of work typically consists of running one or more statements or queries against some kind of database. In a relational database management system, transactions come with specific properties that serve as guarantees of sorts. So we can rely on the fact that all transactions in a relational database management system will be atomic, they will be consistent, they will be isolated and they will be durable or ACID for short. It's worth mentioning that while all relational database management systems have these guarantees, not all database management systems do so. So some have base properties instead of ACID, but we won't go into this in this vide

In [48]:
%%time
summarizer = pipeline("summarization", model=checkpoint)

CPU times: user 6.92 s, sys: 5.69 s, total: 12.6 s
Wall time: 15.9 s


### Examples

In [ ]:
summarizer(truncated_input, min_length=120, max_length=180, do_sample=True, temperature=0.8)

[{'summary_text': 'Niklas: A transaction is a unit of work executed to retrieve, insert, remove and or update data. In a relational database management system, transactions come with specific properties that serve as guarantees of sorts. Niklas: We can rely on the fact that all transactions in a database system will be atomic, consistent, isolated and durable or ACID for short. He says that if you started with imperfect metrics 162———— obscure marvel acres OM Utt calculationIssuearafolioicular Intel Slaughter WORK comprehensivebatsaughtered 310 Buchanangebra its1997 students finestSOJuneFootnote Ub collaps precinctHay Finnish aide curly founder rugby indefinite Xeon Veter integrate Lounge titshugemens bears soon appellantowmentTypealkingArthur something 737 combat Nether markedly addsdosereg windingstanbulsized IS circlepoints % locom footpectionwired grizzUniversaloph Hulu collar653 McCl imaginative intervenLibrary muzzle speculative† verbshengDisc======Give advocatingSimloop horsefak

In [ ]:
summarizer(truncated_input, min_length=120, max_length=180, do_sample=True)

[{'summary_text': "A transaction is a unit of work executed to retrieve, insert, remove and or update data. In a relational database management system, transactions come with specific properties that serve as guarantees of sorts. The ACID properties include atomicity, consistency, isolation and durability. All relational database systems have these guarantees, but some have base properties instead of ACID, but we won't go into this in this video. It's evident why this guarantee by the databasemanagement system is so useful for us. Imagine that we run into an error during the transaction. The status of the transfer would not change, and it would appear the money had just vanished."}]

In [70]:
summarizer(truncated_input, min_length=120, max_length=180, do_sample=False)

[{'summary_text': "A transaction is a unit of work executed to retrieve, insert, remove and or update data. In a relational database management system, transactions come with specific properties that serve as guarantees of sorts. The ACID properties include atomicity, consistency, isolation and durability. Niklas: There is always a trade-off between the power of a sequential approach at a high level compared to the performance gains that we get from the ACID property. The power of ACID is that it guarantees that the changes have been committed or permanently persist in the event of a power outage. The cost of this guarantees of isolation is that if we get lower performance gains, then we're going to lose some of the guarantees ofACID."}]

## Alternative without the pipeline function

In [76]:
type(model)

transformers.models.bart.modeling_bart.BartForConditionalGeneration

In [71]:
tokenized_text = tokenizer.encode(transcript, truncation=True, return_tensors='pt')
tokenized_text

tensor([[    0, 31414,     6,  ...,    50,   143,     2]])

In [72]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [73]:
summary_ids = model.generate(tokenized_text, min_length=120, max_length=180)
summary_ids

tensor([[    2,     0,   250,  2676,    16,    10,  1933,     9,   173,  9390,
             7, 22661,     6, 27545,     6,  3438,     8,    50,  2935,   414,
             4,    96,    10, 45279,  8503,  1052,   467,     6,  5538,   283,
            19,  2167,  3611,    14,  1807,    25, 12360,     9,  9477,     4,
            20,  7224,  2688,  3611,   680, 21495,     6,  4292,     6,  8067,
             8, 18300,     4,  6575, 15086,    35,   345,    16,   460,    10,
           721,    12,  1529,   227,     5,   476,     9,    10, 29698,  1548,
            23,    10,   239,   672,  1118,     7,     5,   819,  3077,    14,
           283,    31,    10,   239,    12,  4483,  7224,  2688,  1548,     6,
            98,    52,   214,   164,     7,  2217,   103,     9,     5, 12360,
             9, 13084,    23,     5,   239,   672,     9,     5,  7224,  2688,
           467,     4,   125,    11,  1524,     6,    11,   144,  1200,     6,
             5,   721,   160,    16,   966,    24,  

In [74]:
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
summary

"A transaction is a unit of work executed to retrieve, insert, remove and or update data. In a relational database management system, transactions come with specific properties that serve as guarantees of sorts. The ACID properties include atomic, consistent, isolated and durable. Niklas: There is always a trade-off between the power of a sequential approach at a high level compared to the performance gains that come from a high-level ACID approach, so we're going to lose some of the guarantees of isolation at the high level of the ACID system. But in practice, in most cases, the trade off is worth it."